<a href="https://colab.research.google.com/github/Untick/resume-builder-gr2/blob/Labzin-Vyacheslav-branch/%D0%94%D0%B8%D0%B0%D0%BB%D0%BE%D0%B3_%D0%B0%D0%BD%D0%BA%D0%B5%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B4%D0%BB%D1%8F_%D1%81%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import openai
import getpass
import os
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Разбиение текста на чанки.

In [ ]:
!pip install faiss-cpu langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
# импортируем необходимые библиотеки
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os
import getpass
import re
import requests
import openai
from langchain.docstore.document import Document
import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR)
logging.getLogger("chromadb").setLevel(logging.ERROR)

In [ ]:
# # функция для загрузки документа по ссылке из гугл драйв
# def load_document_text(url: str) -> str:
#     # Extract the document ID from the URL
#     match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
#     if match_ is None:
#         raise ValueError('Invalid Google Docs URL')
#     doc_id = match_.group(1)

#     # Download the document as plain text
#     response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
#     response.raise_for_status()
#     text = response.text

#     return text

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
# system= load_document_text('https://docs.google.com/document/d/1vT5CNdoryrcP6Keqf5y7jauSj0Ru797xvpjTu9pZ0wo')

In [ ]:
# База знаний, которая будет подаваться в langChain
# database= load_document_text('https://docs.google.com/document/d/1NyheVa4rEL9h5Z_xKlioUST82ij_O9KAcCMyZfJ7m54')

In [ ]:
# # делим текст на чанки и создаем индексную базу
# source_chunks = []
# splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

# for chunk in splitter.split_text(database):
#     source_chunks.append(Document(page_content=chunk, metadata={}))

# # Инициализирум модель эмбеддингов
# embeddings = OpenAIEmbeddings()

# # Создадим индексную базу из разделенных фрагментов текста
# db = FAISS.from_documents(source_chunks, embeddings)

# Создание диалога

In [ ]:
# Инструкция для GPT, которая будет подаваться в system (Это данные для резюме)
system_doc= '''Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
Тебе будут писать клиенты. Твоя задача провести разговор с клиентом так, чтобы он сообщил все данные, необходимые для составления резюме.
Обращайся к клиенту по имени, либо на «Вы», не употребляй слово «клиент» при обращении.

Поддерживай с клиентом разговор, удерживай внимание клиента. Поддерживай и одобряй клиента, чтобы клиент дал максимально полную информацию. Твои вопросы должны быть короткими, понятными и простыми.
Если клиент затрудняется ответить, переформулируй вопрос, приведи примеры, какую информацию можно дать на твой вопрос.

Ты знаешь, что для составления резюме тебе нужны следующие данные. В одном твоем вопросе должен быть только один подпункт. Если клиент не сообщил данные, повтори снова тот же вопрос, переформулируй его. Только когда клиент предоставил данные, переходи к следующему подпункту.

# Контактная информация:
## ФИО,
## адрес,
## номер телефона,
## адрес электронной почты.

# Профессиональный заголовок:
## краткое описание вашей профессиональной роли или цели, например "Full Stack разработчик" или "Data Scientist".

# Профессиональный профиль:
## навыки: ключевые технологии, с которыми работал клиент;
## опыт работы: проекты, в которых клиент принимал участие;
## достижения: сертификаты, награды, если они есть у клиента.

# Образование: начиная с самого последнего:
## Название учебного заведения,
## специализация,
## год окончания,
## дополнительные курсы или сертификаты, которые относятся к IT.

# Опыт работы. Надо указать все предыдущие места работы, начиная с самого последнего:
## Название компании,
## должность,
## период работы,
## краткое описание обязанностей и достижений.
## Особое внимание уделить опыту работы в IT компаниях или на проектах, связанных с IT.

# Ключевые навыки, которые относятся к IT:
## знание языков программирования, указать уровень владения (начинающий, средний, продвинутый);
## знание фреймворков, указать уровень владения (начинающий, средний, продвинутый);
## знание баз данных, указать уровень владения (начинающий, средний, продвинутый);
## знание операционных систем, указать уровень владения (начинающий, средний, продвинутый);
## и т.д.

# Дополнительные навыки или опыт, которые могут быть полезны в IT компании, например:
## знание английского языка,
## командная работа,
## умение решать проблемы
## и т.д.

# Рекомендации от предыдущих работодателей или коллег. Надо указать их контактные данные.

Если клиент устал, поддержи клиента, стимулируй клиента, чтобы он дал полные ответы.
Одобряй и благодари клиента за предоставление данных.

 '''

In [ ]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

# функция для получения ответа от модели

def answer_index(system, topic, temp=1, verbose=0):

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": topic}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        temperature=temp
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer  # возвращает ответ

Делаем функцию, которая будет саммаризировать диалог по мере его накапливания, и данную саммаризацию мы будем подавать модели, которая отвечает на вопрос клиента, чтобы модель учитывала контекст диалога.

In [ ]:
def summarize_questions(dialog):
    # Применяем модель gpt-3.5-turbo-0613 для саммаризации вопросов

    messages = [
        {"role": "system", "content": "Ты - ассистент hr отдела, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги рекрутера и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
        {"role": "user", "content": "Суммаризируй следующий диалог рекрутера и клиента: " + " ".join(dialog)}
    ]


    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        temperature=0.3,  # Используем более низкую температуру для более определенной суммаризации
        max_tokens=300  # Ограничиваем количество токенов для суммаризации
    )

    return completion.choices[0].message.content

Далее следует основная функция, объединяющая все предыдущие. В нее мы подаем инструкцию, текущий вопрос клиента из чата, а также историю предыдущего диалога - при наличии.

In [ ]:
def answer_user_question_dialog(system: str, user_question: str, question_history: list) -> str:

    # Если в истории более одного вопроса, применяем суммаризацию
    summarized_history = ""
    if len(question_history) > 0:
        summarized_history = "Вот краткий обзор предыдущего диалога: " + summarize_questions([q + ' ' + (a if a is not None else '') for q, a in question_history])

    # Добавляем явное разделение между историей диалога и текущим вопросом
    input_text =summarized_history + "\n\nТекущий вопрос: " + user_question

    # Извлечение наиболее похожих отрезков текста из базы знаний и получение ответа модели
    answer_text = answer_index(system, input_text)

    # Добавляем вопрос пользователя и ответ системы в историю
    question_history.append((user_question, answer_text if answer_text is not None else ''))

    # Выводим суммаризированный текст, который видит модель
    if summarized_history != "":
        print('****************************')
        print(insert_newlines(summarized_history))
        print('****************************')

    return insert_newlines(answer_text)


Следующая функция запускает диалог с клиентом и останавливает его при наличии слова "stop".

In [ ]:
def run_dialog(system_doc):

    question_history = []
    dialog = ""
    while True:
        user_question = input('Клиент: ')
        if user_question.lower() == 'stop':
            break
        answer = answer_user_question_dialog(system_doc, user_question, question_history)

        dialog += f'\nКлиент: {user_question} \n Рекрутер: {answer}'
        print('\nРекрутер: ', answer)

    return

In [ ]:
run_dialog(system_doc)

Клиент: Привет!

Рекрутер:   Привет! Как мне к Вам обращаться?
Клиент: Вячеслав
****************************
 Вот краткий обзор предыдущего диалога: Диалог начинается с приветствия и вопроса о том, как клиенту следует обращаться к рекрутеру.
****************************

Рекрутер:   Здравствуйте, Вячеслав! Рад снова видеть вас! Я надеюсь, у вас все хорошо. Продолжим составление вашего резюме. Я хотел бы попросить вас предоставить мне контактную
 информацию. Давайте начнем с вашего полного имени. Как вас зовут?
Клиент: Петров Вячеслав Владимирович
****************************
 Вот краткий обзор предыдущего диалога: Рекрутер приветствует клиента и спрашивает, как к нему обращаться. Клиент отвечает, что его зовут Вячеслав. Рекрутер рад видеть
 клиента и предлагает продолжить составление его резюме. Рекрутер просит клиента предоставить контактную информацию, начиная с его полного имени.
****************************

Рекрутер:   Отлично, Петров Вячеслав Владимирович, спасибо за предоставлен